<a href="https://colab.research.google.com/github/torimonn/disclosure/blob/main/%E8%B2%B8%E5%80%9F%E5%AF%BE%E7%85%A7%E8%A1%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==============================================================================
# 1. 必要なライブラリのインストール
# ==============================================================================
# google-genai: Google Generative AI SDKの最新版
# pydantic: 構造化出力のためのデータモデル定義
# pypdf: (補助的に) 環境によってはPDFのテキストを明示的にプロンプトに含めると、
#        精度が向上する場合があるため、テキスト抽出用にインストールします。
#        PDFの直接入力と併用します。
!pip install -qU google-genai pydantic pypdf

In [2]:
# ==============================================================================
# 2. インポート＆環境変数の設定（APIキー）
# ==============================================================================
import os
import json
import time
from datetime import date
from typing import List, Optional, Union

# Google Colab専用のモジュール。Colab以外では不要ですが、このままでもエラーにはなりません。
from google.colab import userdata
import google.colab.files  # Colab上でファイルアップロードUIを表示するために使います

# Google Generative AI SDKをインポート
import google.generativeai as genai

# Pydanticを使ってデータ構造（JSONスキーマ）を定義する
from pydantic import BaseModel, Field, ValidationError

# PyPDFを使ってPDFのテキストを抽出する
from pypdf import PdfReader

# CSV 出力用に pandas をインポート
import pandas as pd

# ----------------------------------------------------------------------------
# Google Gemini（Vertex AI）API キーの設定
# ----------------------------------------------------------------------------
try:
    # Colab の Secrets から取得する例
    google_api_key = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=google_api_key)
    print("APIキーの設定が完了しました。")
except userdata.SecretNotFoundError as e:
    print(f"エラー: Colab Secrets に 'GOOGLE_API_KEY' が見つかりません: {e}")
    print("Google Colab の Secrets に 'GOOGLE_API_KEY' を設定してください。")
    exit()
except Exception as e:
    print(f"APIキー設定中に予期せぬエラーが発生しました: {e}")
    exit()


APIキーの設定が完了しました。


In [3]:
# ==============================================================================
# 3. Pydantic モデル定義
# ==============================================================================

# ----------------------------------------------------------------------------
# 3-1. PDF メタデータ抽出用モデル
# ----------------------------------------------------------------------------
class PDFMetadata(BaseModel):
    """
    PDF文書から抽出する基本メタデータを表すモデル。
    - company_name_japanese: 会社名（日本語）
    - company_name_english: 会社名（英語。なければ None）
    - balance_sheet_pages_1_indexed: 貸借対照表が掲載されているページ番号リスト（1 から始まる）
    - estimated_balance_sheet_type: 貸借対照表の種類推定（例: '単体','連結','両方','不明'）
    """
    company_name_japanese: str = Field(description="会社名（日本語）")
    company_name_english: Optional[str] = Field(default=None, description="会社名（英語）。なければ null")
    balance_sheet_pages_1_indexed: List[int] = Field(description="貸借対照表のページ番号リスト（1-indexed）")
    estimated_balance_sheet_type: Optional[str] = Field(default=None, description="貸借対照表の種類推定")

# ----------------------------------------------------------------------------
# 3-2. 貸借対照表抽出用モデル（再帰的に子項目を持つモデルなど）
# ----------------------------------------------------------------------------
class BalanceSheetItem(BaseModel):
    """
    貸借対照表の個別項目を表すモデル。
    - name_japanese: 勘定科目名（日本語）
    - name_english: 勘定科目名（英語）
    - value: 金額（円）。データがなければ None。
    - indent_level: 階層レベル（0: トップレベル、1: 子項目、…）
    - children: 子項目リスト（再帰的に BalanceSheetItem）
    """
    name_japanese: str = Field(description="勘定科目名（日本語）")
    name_english: str = Field(description="勘定科目名（英語）")
    value: Optional[Union[int, float]] = Field(default=None, description="金額（円）。なければ null")
    indent_level: int = Field(description="階層レベル（0: トップ、1: 子項目、…）", ge=0)
    children: List['BalanceSheetItem'] = Field(default_factory=list, description="子項目リスト（再帰的）")

# 再帰参照を解決するため
BalanceSheetItem.update_forward_refs()

class BalanceSheetSection(BaseModel):
    """
    貸借対照表の主要セクションを表すモデル。
    - total_value: セクション合計金額（円）。不明なら None。
    - items: セクション内の勘定科目リスト
    """
    total_value: Optional[Union[int, float]] = Field(default=None, description="セクション合計金額（円）。不明なら null")
    items: List[BalanceSheetItem] = Field(description="セクション内の勘定科目リスト")

class FiscalYearBalanceSheet(BaseModel):
    """
    ある会計年度分の貸借対照表を表すモデル。
    - end_date: 会計年度末日 (YYYY-MM-DD 形式)
    - description: 会計年度の補足情報（例: '2022年度 (2023/3末)'）
    - assets, liabilities, net_assets: 各セクションのデータ
    """
    end_date: date = Field(description="会計年度末日 (YYYY-MM-DD)")
    description: Optional[str] = Field(default=None, description="会計年度の補足情報")
    assets: BalanceSheetSection = Field(description="資産の部")
    liabilities: BalanceSheetSection = Field(description="負債の部")
    net_assets: BalanceSheetSection = Field(description="純資産の部")

# ----------------------------------------------------------------------------
# 3-3. LLM の出力（トップレベルに "balance_sheet" がある）に合わせるためのモデル
# ----------------------------------------------------------------------------
class BalanceSheetBody(BaseModel):
    """
    LLM が返す JSON ではトップレベルに "balance_sheet" キーがあり、その中に fiscal_year_data 配列がある。
    ここではその中身部分を表すモデル。
    """
    fiscal_year_data: List[FiscalYearBalanceSheet] = Field(description="複数会計年度の貸借対照表データ")

class BalanceSheetResponse(BaseModel):
    """
    LLM が返す JSON 全体を受け取るモデル。
    トップレベルに "balance_sheet" キーを持つ想定。
    """
    balance_sheet: BalanceSheetBody

print("Pydantic モデルの定義が完了しました。\n")


Pydantic モデルの定義が完了しました。



<ipython-input-3-26ce75f332f9>:40: PydanticDeprecatedSince20: The `update_forward_refs` method is deprecated; use `model_rebuild` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  BalanceSheetItem.update_forward_refs()


In [4]:
# ==============================================================================
# 4. PDF コンテンツ準備関数／テキスト抽出関数
# ==============================================================================
def upload_pdf_to_genai(pdf_file_path: str) -> Optional[genai.protos.FileData]:
    """
    PDF ファイルを Google Generative AI サービスにアップロードし、FileData オブジェクトを返す。
    - pdf_file_path: ローカルにある PDF ファイルパス。
    """
    try:
        uploaded_file = genai.upload_file(
            path=pdf_file_path,
            display_name=os.path.basename(pdf_file_path)
        )
        print(f"ファイル '{uploaded_file.display_name}' を GenAI サービスにアップロードしました。")
        print(f"URI: {uploaded_file.uri}")

        # 処理が終わるまで待機
        while uploaded_file.state.name == 'PROCESSING':
            print("ファイル処理中… 10秒待機します。")
            time.sleep(10)
            uploaded_file = genai.get_file(uploaded_file.name)

        if uploaded_file.state.name == 'FAILED':
            print(f"ファイルの処理に失敗しました: {uploaded_file.name}")
            return None

        print(f"ファイル処理完了。ステータス: {uploaded_file.state.name}")
        return uploaded_file

    except Exception as e:
        print(f"PDF のアップロード中にエラーが発生しました: {e}")
        return None

def extract_text_from_pdf_pypdf(pdf_file_path: str, page_numbers: Optional[List[int]] = None) -> str:
    """
    PyPDF を使って PDF から指定ページのテキストを抽出し、文字列として返す。
    - page_numbers: 0-indexed のページ番号リスト。None または空リストなら全ページから抽出。
    """
    try:
        reader = PdfReader(pdf_file_path)
        extracted_text = []

        if page_numbers:
            for page_num in page_numbers:
                if 0 <= page_num < len(reader.pages):
                    extracted_text.append(reader.pages[page_num].extract_text() or "")
        else:
            for page in reader.pages:
                extracted_text.append(page.extract_text() or "")

        return "\n".join(extracted_text)
    except Exception as e:
        print(f"PyPDF によるテキスト抽出中にエラーが発生しました: {e}")
        return ""


In [5]:
# ==============================================================================
# 5. LLM への指示テキスト生成関数（メタデータ抽出用）
# ==============================================================================
def create_generative_content_parts_for_metadata(
    uploaded_pdf_file: genai.protos.FileData,
    pydantic_schema: dict
) -> List[Union[str, genai.protos.FileData]]:
    """
    メタデータ抽出のために LLM に渡すコンテンツを作成する。
    - uploaded_pdf_file: アップロード済みの PDF ファイルオブジェクト
    - pydantic_schema: PDFMetadata.schema() などの JSON スキーマ dict
    """
    content_parts = []
    schema_json = json.dumps(pydantic_schema, indent=2, ensure_ascii=False)

    instruction_text = """
提供された PDF 文書全体を分析し、以下の情報を抽出してください。
出力は必ず指定の JSON スキーマの形式のみとし、追加のテキストや説明は含めないでください。

### 抽出指示:
1. **会社名（日本語）**: PDF の表紙や目次から信用金庫の正式名称（日本語）を抽出してください。
2. **会社名（英語）**: 英語の名称があれば抽出し、なければ null。
3. **貸借対照表のページ番号**: 目次や財務諸表セクションを参照し、貸借対照表が掲載されているページ番号を 1 から始まるインデックスでリストにしてください。
4. **貸借対照表の種類**: 単体、連結、両方のいずれかを推測し、なければ「不明」としてください。

### JSON スキーマ:
{json_schema_str}
"""
    # {json_schema_str} 部分を置換
    filled_instruction = instruction_text.format(json_schema_str=schema_json)
    content_parts.append(filled_instruction)
    content_parts.append(uploaded_pdf_file)
    return content_parts

In [6]:
# ==============================================================================
# 6. LLM への指示テキスト生成関数（貸借対照表抽出用）
# ==============================================================================
def create_generative_content_parts_for_balance_sheet(
    uploaded_pdf_file: genai.protos.FileData,
    company_name_japanese: str,
    company_name_english: str,
    pydantic_schema: dict,
    auxiliary_pdf_text: Optional[str] = None
) -> List[Union[str, genai.protos.FileData]]:
    """
    貸借対照表抽出のために LLM に渡すコンテンツを作成する。
    - uploaded_pdf_file: アップロード済みの PDF ファイルオブジェクト
    - company_name_japanese, company_name_english: メタデータ抽出で得られた会社名
    - pydantic_schema: BalanceSheetResponse 用の JSON スキーマ dict（定義をユーザーが作成済み）
    - auxiliary_pdf_text: PyPDF で抽出した補助テキスト（省略可）
    """
    content_parts = []
    schema_json = json.dumps(pydantic_schema, indent=2, ensure_ascii=False)

    instruction_text = """
以下の会社「{company_jp} ({company_en})」のディスクロージャー PDF 文書全体を分析し、
貸借対照表データを指定の JSON スキーマ形式で抽出してください。

### 抽出指示:
1. **PDF 文書全体を分析**: 提供された PDF の全ページを解析し、貸借対照表の位置を特定してください。
2. **会計年度データ**: PDF に記載されている全ての会計年度の貸借対照表データを抽出してください。
   - `end_date`: 会計年度末日を "YYYY-MM-DD" 形式で正確に抽出。
   - `description`: 表に書かれている年度説明（例: "2022年度 (2023/3末)"）。なければ null。複数の期が表示されている場合は最新の年度を抽出する。
   - `sections`:
       * `assets`: 資産の部。合計金額と勘定科目を抽出。
       * `liabilities`: 負債の部。合計金額と勘定科目を抽出。
       * `net_assets`: 純資産の部。合計金額と勘定科目を抽出。
   - 各セクション内の `items`:
       * `name_japanese`: 勘定科目名（日本語）。
       * `name_english`: 勘定科目名（英語）。わからなければローマ字でも可。
       * `value`: 金額（整数または小数）。単位は「円」、マイナスは負の数。データなしや "-" は null。
       * `indent_level`: 字下げに基づく階層レベル（0: トップ、1: 子項目、…）。
       * `children`: 子項目があればリストに含める。
   - 各セクションの合計金額は `total_value` に入れる。

3. **出力は JSON のみ**: 追加テキストや説明を含めないでください。

### JSON スキーマ:
{json_schema_str}
"""
    filled_instruction = instruction_text.format(
        company_jp=company_name_japanese,
        company_en=company_name_english,
        json_schema_str=schema_json
    )
    content_parts.append(filled_instruction)
    content_parts.append(uploaded_pdf_file)
    if auxiliary_pdf_text:
        content_parts.append("\n\n### 補助テキスト（PDFから抽出したテキスト）:\n" + auxiliary_pdf_text)
    return content_parts

In [7]:
# ==============================================================================
# 7. LLM 呼び出し＆構造化抽出関数
# ==============================================================================
def call_llm_for_structured_output(
    content_parts: List[Union[str, genai.protos.FileData]],
    output_model: BaseModel,
    model_name: str = "gemini-2.5-flash-preview-05-20"  # 利用可能なモデル名を list_models() で確認して指定
) -> Optional[BaseModel]:
    """
    マルチモーダル入力（テキスト + PDF）を LLM に渡し、指定の Pydantic モデルでバリデーションした結果を返す。
    - content_parts: テキストと FileData オブジェクトのリスト
    - output_model: Pydantic モデルのクラス（例: PDFMetadata, BalanceSheetResponse）
    - model_name: 使用するモデル名（必ず ListModels でサポートを確認すること）
    """
    try:
        model = genai.GenerativeModel(model_name=model_name)
        response = model.generate_content(
            contents=content_parts,
            generation_config=genai.GenerationConfig(
                response_mime_type="application/json",
                temperature=0
            ),
            safety_settings={
                'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE',
                'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE',
                'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE',
                'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE',
            }
        )

        response_text = response.text.strip()
        # ```json で囲まれている場合の対応
        if response_text.startswith("```json") and response_text.endswith("```"):
            response_text = response_text[7:-3].strip()

        parsed_json = json.loads(response_text)
        validated_data = output_model.parse_obj(parsed_json)
        return validated_data

    except ValidationError as e:
        print("Pydantic バリデーションエラーが発生しました:")
        print(e.json())
        print("LLM の生の応答 (バリデーション失敗):")
        print(response_text)
        return None

    except json.JSONDecodeError as e:
        print("LLM の応答が不正な JSON 形式です:")
        print(e)
        print("LLM の生の応答 (JSON パース失敗):")
        print(response_text)
        return None

    except Exception as e:
        print(f"LLM による抽出中に予期せぬエラーが発生しました: {e}")
        return None

In [11]:
# ==============================================================================
# 8. CSV 出力用関数
# ==============================================================================
def export_balance_sheet_to_csv(
    balance_sheet_resp: BalanceSheetResponse,
    company_name_japanese: str,
    output_path: str = "balance_sheet.csv"
) -> None:
    """
    BalanceSheetResponse から CSV ファイルを作成する。
    - balance_sheet_resp: LLM から得た BalanceSheetResponse オブジェクト
    - company_name_japanese: PDF メタデータ抽出で得た会社名（日本語）
    - output_path: 保存する CSV ファイルのパス
    """
    # '最新' 会計年度データを取得（インデックス 0）
    latest = balance_sheet_resp.balance_sheet.fiscal_year_data[0]

    # 再帰的に子項目を展開して「(勘定科目, 金額)」タプルを集める
    def flatten_items(items, rows):
        for item in items:
            rows.append((item.name_japanese, item.value if item.value is not None else ""))
            if item.children:
                flatten_items(item.children, rows)

    rows = []
    # 資産の部
    flatten_items(latest.assets.items, rows)
    # 負債の部
    flatten_items(latest.liabilities.items, rows)
    # 純資産の部
    flatten_items(latest.net_assets.items, rows)

    # --------------------------------------------------------------------------------
    # CSV 用のデータを作成する
    # 1行目: A 列は空、B 列に会社名
    # 2行目: A 列は空、B 列に「会計年度末日」を入れる
    # 3行目以降: A 列に勘定科目、B 列に金額
    # --------------------------------------------------------------------------------
    data = []
    # 1行目
    data.append(("", company_name_japanese))
    # 2行目 ― 会計年度末日を B 列へ
    data.append(("", latest.end_date.strftime("%Y-%m-%d")))
    # 3行目以降
    for subj, val in rows:
        data.append((subj, val))

    df = pd.DataFrame(data, columns=["勘定科目", "金額"])

    # CSV に保存
    df.to_csv(output_path, index=False, encoding="utf-8-sig")
    print(f"CSV を保存しました: {output_path}")

In [12]:
# ==============================================================================
# 9. メイン処理
# ==============================================================================
if __name__ == "__main__":
    uploaded_pdf_local_path = None

    # Colab のアップロード UI を表示
    print("貸借対照表を抽出したい PDF ファイルをアップロードしてください。\n")
    uploaded = google.colab.files.upload()

    if uploaded:
        # 最初にアップロードされたファイルを使う
        for filename in uploaded.keys():
            uploaded_pdf_local_path = os.path.join("/content/", filename)
            print(f"'{filename}' を '/content/' にアップロードしました。")
            break
    else:
        print("PDF ファイルがアップロードされませんでした。スクリプトを終了します。")
        exit()

    # --------------------------------------------------------
    # 9-1. PDF メタデータ抽出
    # --------------------------------------------------------
    print(f"\n'{uploaded_pdf_local_path}' を GenAI サービスにアップロード中（メタデータ抽出用）...")
    uploaded_pdf_file_for_metadata = upload_pdf_to_genai(uploaded_pdf_local_path)

    if not uploaded_pdf_file_for_metadata:
        print("メタデータ抽出用のアップロード・処理に失敗しました。終了します。")
        try:
            os.remove(uploaded_pdf_local_path)
        except:
            pass
        exit()

    print("\n--- LLM による PDF メタデータの特定を開始します ---")
    pdf_metadata_schema = PDFMetadata.schema()
    metadata_content_parts = create_generative_content_parts_for_metadata(
        uploaded_pdf_file=uploaded_pdf_file_for_metadata,
        pydantic_schema=pdf_metadata_schema
    )

    pdf_metadata: Optional[PDFMetadata] = call_llm_for_structured_output(
        content_parts=metadata_content_parts,
        output_model=PDFMetadata
    )

    # アップロードしたファイルを削除してクリーンアップ
    try:
        genai.delete_file(uploaded_pdf_file_for_metadata.name)
    except:
        pass

    if not pdf_metadata:
        print("PDF メタデータの特定に失敗しました。スクリプトを終了します。")
        try:
            os.remove(uploaded_pdf_local_path)
        except:
            pass
        exit()

    print("\n--- PDF メタデータ特定成功 ---")
    print(f"会社名（日本語）: {pdf_metadata.company_name_japanese}")
    print(f"会社名（英語）: {pdf_metadata.company_name_english}")
    print(f"貸借対照表ページ（1-indexed）: {pdf_metadata.balance_sheet_pages_1_indexed}")
    print(f"貸借対照表の種類推定: {pdf_metadata.estimated_balance_sheet_type}")

    # メタデータから会社名を取得
    target_company_japanese = pdf_metadata.company_name_japanese
    target_company_english = pdf_metadata.company_name_english or target_company_japanese

    # ページ番号を 0-indexed に変換（PyPDF 用）
    if pdf_metadata.balance_sheet_pages_1_indexed:
        pages_for_auxiliary_text_0_indexed = [p - 1 for p in pdf_metadata.balance_sheet_pages_1_indexed]
    else:
        pages_for_auxiliary_text_0_indexed = None

    # --------------------------------------------------------
    # 9-2. 貸借対照表抽出
    # --------------------------------------------------------
    print(f"\n'{uploaded_pdf_local_path}' を GenAI サービスにアップロード中（貸借対照表抽出用）...")
    uploaded_pdf_file_for_bs = upload_pdf_to_genai(uploaded_pdf_local_path)

    if not uploaded_pdf_file_for_bs:
        print("貸借対照表抽出用のアップロード・処理に失敗しました。スクリプトを終了します。")
        try:
            os.remove(uploaded_pdf_local_path)
        except:
            pass
        exit()

    # 補助テキスト抽出（省略可）
    print("\nPDF から補助テキストを抽出中（LLM の精度向上のため）...")
    auxiliary_pdf_text = extract_text_from_pdf_pypdf(
        uploaded_pdf_local_path,
        pages_for_auxiliary_text_0_indexed
    )

    print("\n--- LLM による貸借対照表の構造化抽出を開始します ---")
    # ここではユーザーが定義した JSON スキーマ（BalanceSheetResponse 用）を dict 形式で渡す
    pydantic_schema_bs = BalanceSheetResponse.schema()

    content_parts_bs = create_generative_content_parts_for_balance_sheet(
        uploaded_pdf_file=uploaded_pdf_file_for_bs,
        company_name_japanese=target_company_japanese,
        company_name_english=target_company_english,
        pydantic_schema=pydantic_schema_bs,
        auxiliary_pdf_text=auxiliary_pdf_text
    )

    balance_sheet_resp: Optional[BalanceSheetResponse] = call_llm_for_structured_output(
        content_parts=content_parts_bs,
        output_model=BalanceSheetResponse
    )

    # 貸借対照表抽出用ファイルを削除してクリーンアップ
    try:
        genai.delete_file(uploaded_pdf_file_for_bs.name)
    except:
        pass

    if not balance_sheet_resp:
        print("貸借対照表データの抽出に失敗しました。スクリプトを終了します。")
        try:
            os.remove(uploaded_pdf_local_path)
        except:
            pass
        exit()

    print("\n--- 貸借対照表抽出成功 ---")
    # 抽出された会計年度データの例を表示
    latest_year = balance_sheet_resp.balance_sheet.fiscal_year_data[0]
    print(f"最新会計年度末日: {latest_year.end_date}")
    print(f"資産合計: {latest_year.assets.total_value} 円")
    print(f"負債合計: {latest_year.liabilities.total_value} 円")
    print(f"純資産合計: {latest_year.net_assets.total_value} 円\n")

    # --------------------------------------------------------
    # 9-3. CSV 出力
    # --------------------------------------------------------
    csv_output_path = "/content/balance_sheet.csv"
    export_balance_sheet_to_csv(
        balance_sheet_resp=balance_sheet_resp,
        company_name_japanese=target_company_japanese,
        output_path=csv_output_path
    )

    # Colab 上で自動的にダウンロードを開始する
    try:
        from google.colab import files
        files.download(csv_output_path)
        print(f"CSV をダウンロード用に準備しました: {csv_output_path}")
    except Exception as e:
        print(f"CSV のダウンロード中にエラーが発生しました: {e}")

    # ローカルの PDF ファイルを削除して終了
    try:
        os.remove(uploaded_pdf_local_path)
        print(f"ローカルファイル '{uploaded_pdf_local_path}' を削除しました。")
    except Exception as e:
        print(f"ローカルファイル削除中にエラーが発生しました: {e}")

貸借対照表を抽出したい PDF ファイルをアップロードしてください。



Saving disclo_2024keisu.pdf to disclo_2024keisu (2).pdf
'disclo_2024keisu (2).pdf' を '/content/' にアップロードしました。

'/content/disclo_2024keisu (2).pdf' を GenAI サービスにアップロード中（メタデータ抽出用）...
ファイル 'disclo_2024keisu (2).pdf' を GenAI サービスにアップロードしました。
URI: https://generativelanguage.googleapis.com/v1beta/files/ww1d5rkbz0y3
ファイル処理完了。ステータス: ACTIVE

--- LLM による PDF メタデータの特定を開始します ---


<ipython-input-12-7cf4e1d1745b>:36: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  pdf_metadata_schema = PDFMetadata.schema()
<ipython-input-7-4ab6ee6d0491>:37: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  validated_data = output_model.parse_obj(parsed_json)



--- PDF メタデータ特定成功 ---
会社名（日本語）: 城南信用金庫
会社名（英語）: None
貸借対照表ページ（1-indexed）: [26, 27, 54, 55]
貸借対照表の種類推定: 両方

'/content/disclo_2024keisu (2).pdf' を GenAI サービスにアップロード中（貸借対照表抽出用）...
ファイル 'disclo_2024keisu (2).pdf' を GenAI サービスにアップロードしました。
URI: https://generativelanguage.googleapis.com/v1beta/files/3z168m9ipcv3
ファイル処理完了。ステータス: ACTIVE

PDF から補助テキストを抽出中（LLM の精度向上のため）...

--- LLM による貸借対照表の構造化抽出を開始します ---


<ipython-input-12-7cf4e1d1745b>:100: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  pydantic_schema_bs = BalanceSheetResponse.schema()



--- 貸借対照表抽出成功 ---
最新会計年度末日: 2023-03-31
資産合計: 4220215356273 円
負債合計: 4034361995925 円
純資産合計: 185853360348 円

CSV を保存しました: /content/balance_sheet.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CSV をダウンロード用に準備しました: /content/balance_sheet.csv
ローカルファイル '/content/disclo_2024keisu (2).pdf' を削除しました。
